In [1]:
import sys
sys.stdout.flush()

import flwr as fl
from flwr.server.strategy import FedAvg
import torch
import torch.nn as nn
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.all import *
from task import plotar_graficos
import numpy as np
import json
from sklearn.metrics import confusion_matrix, roc_curve, auc

model = nn.Sequential(create_body(xresnet50(), pretrained=False), create_head(nf=2048, n_out=2))
acc_h = []
loss_h = []
predicted_labels_h = []
true_labels_h = []
preds_h = []

def get_parameters(net):
    return [val.cpu().detach().numpy() for _, val in net.state_dict().items()]

def weighted_average(metrics):
    
    aggregated_metrics = {}
    metric_keys = metrics[0][1].keys()

    for key in metric_keys:
        if key in ["predicted_labels", "true_labels", "preds"]:
            aggregated_metrics[key] = json.dumps(
                sum((json.loads(metric[key]) for _, metric in metrics), [])
            )
        else:
            total = sum(num_examples * metric[key] for num_examples, metric in metrics)
            aggregated_metrics[key] = total / sum(num_examples for num_examples, _ in metrics)            

    # print(aggregated_metrics)
    if aggregated_metrics != {}:
        acc_h.append(aggregated_metrics['accuracy'])
        loss_h.append(aggregated_metrics['loss'])

        predicted_labels_h.append(np.array(json.loads(aggregated_metrics["predicted_labels"])))
        true_labels_h.append(np.array(json.loads(aggregated_metrics["true_labels"])))
        preds_h.append(np.array(json.loads(aggregated_metrics["preds"])))
    
    return aggregated_metrics

def main():
    server_address = "0.0.0.0:8080"
    initial_parameters = get_parameters(model)
    strategy = FedAvg(
        initial_parameters=fl.common.ndarrays_to_parameters(initial_parameters),
        fit_metrics_aggregation_fn=weighted_average,
        evaluate_metrics_aggregation_fn=weighted_average,
    )
    fl.server.start_server(
        server_address=server_address,
        config=fl.server.ServerConfig(num_rounds=5),
        strategy=strategy,
    )


if __name__ == "__main__":
    main()    

	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=5, no round_timeout
INFO :      Flower ECE: gRPC server running (5 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      Evaluation returned no results (`None`)
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 2 clients (out of 2)
INFO :      aggregate_fit: received 2 results and 0 failures
INFO :      configure_evaluate: strategy sampled 2 clients (out of 2)
INFO :      aggregate_evaluate: received 2 results and 0 failures
I